In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Data/sales_predictions (3).csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# Exploring Data

In [3]:
# How many rows and columns?
df.shape

(8523, 12)

In [4]:
# display the information of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


# Data Cleaning

## Drop unnecessary columns or rows

In [5]:
# Drop any duplicates 
df.duplicated().sum() # no duplicates

0

In [6]:
## Drop unnecessary columns
df = df.drop(columns = ['Item_Identifier', 'Outlet_Identifier', 'Outlet_Establishment_Year'])

There are three columns that don't seem to make contributions to the food sales, which are 'Item_Identifier', 'Outlet_Identifier', 'Outlet_Establishment_Year'.

so I decided to drop them.

## Addressing Missing Values

In [7]:
# Identify missing values
df.isna().sum()

Item_Weight             1463
Item_Fat_Content           0
Item_Visibility            0
Item_Type                  0
Item_MRP                   0
Outlet_Size             2410
Outlet_Location_Type       0
Outlet_Type                0
Item_Outlet_Sales          0
dtype: int64

we can see that there are 1463 missing values in column 'item weight' and 2410 missing values in column'outlet size'.

next let's figure out how to address the missing values.

In [8]:
# display the stats information of the num columns.
df.describe()


,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,2181.288914
std,4.643456,0.051598,62.275067,1706.499616
min,4.555000,0.000000,31.290000,33.290000
25%,8.773750,0.026989,93.826500,834.247400
50%,12.600000,0.053931,143.012800,1794.331000
75%,16.850000,0.094585,185.643700,3101.296400
max,21.350000,0.328391,266.888400,13086.964800


we can see that the average weight of the items is around 12.86 and half of the items weight at 12.6. 

so, I can decide to use the mean of the weight to fill the missing values.

In [9]:
# address the missing values in item-weight.
weight_mean = df['Item_Weight'].mean()
df['Item_Weight'].fillna(weight_mean, inplace=True)
# double check the missing values
df.isna().sum()

Item_Weight                0
Item_Fat_Content           0
Item_Visibility            0
Item_Type                  0
Item_MRP                   0
Outlet_Size             2410
Outlet_Location_Type       0
Outlet_Type                0
Item_Outlet_Sales          0
dtype: int64

In [10]:
# addressing missing values in outlet_size.
# check the values counts in outlet-size
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

we can see that most of the outlet size is medium size.
so I can decide to fill the missing values use the medium size. 

In [11]:
df['Outlet_Size'].fillna('Medium', inplace=True)
# double check the missing values
df.isna().sum()

Item_Weight             0
Item_Fat_Content        0
Item_Visibility         0
Item_Type               0
Item_MRP                0
Outlet_Size             0
Outlet_Location_Type    0
Outlet_Type             0
Item_Outlet_Sales       0
dtype: int64

## Find and fix any inconsistent categories of data

In [12]:
# filter out all the object dtype columns
dtypes = df.dtypes
cat_cols = dtypes[dtypes=="object"].index
for col in cat_cols:
  print(f'-Column={col}')
  print(df[col].value_counts(dropna=False))
  print('\n\n')

-Column=Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



-Column=Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



-Column=Outlet_Size
Medium    5203
Small     2388
High       932
Name: Outlet_Size, dtype: int64



-Column=Outlet_Location_Type
Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64



-Column=Outlet_Type
Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermar

we can see that in the 'Item_Fat_Content' column; there are actually only two types of fat: low fat and regular. 

so we are going to replace the LF, low fat with Low Fat and reg with Regular.

In [13]:
# address the inconsistence in Item_Fat_Content column.
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF': 'Low Fat',
                                                        'low fat' : 'Low Fat',
                                                        'reg' : 'Regular'})

In [14]:
# double the value counts of 'Item_Fat_Content'
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64